In [4]:
# Headers
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from copy import deepcopy
from skimage.feature import greycomatrix, greycoprops
import cv2  
from numpy.linalg import det
from skimage import util, exposure
from math import sqrt, ceil
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io

In [5]:
# global variables
black_value = np.float64(-1408.5106382978724)
images_filename = 'tr_im.nii'
masks_filename = 'tr_mask.nii'

In [6]:
def get_vals(mask):
    vals = []
    x, y = mask.shape
    for i in range(x):
        for j in range(y):
            if mask[i,j] not in vals:
                vals.append(mask[i][j])

    vals.remove(0)
    return vals

In [7]:
def apply_mask(img, mask, class_id):
    heigh, width = img.shape
    tmp = deepcopy(img)
    lineal_array = np.array(0)
    for x in range(heigh):
        for y in range(width):
            # Set pixel as black color if is not inside this mask
            if mask[x,y] != np.float64(class_id):
                tmp[x,y] = black_value
            # Count pixels inside this mask
            else:
                lineal_array = np.append(lineal_array, tmp[x,y])
    # In tmp is saved the image with mask applied
    # In lineal_array is saved just the mask's pixels
    lineal_array = np.delete(lineal_array, 0)
    return tmp, lineal_to_matrix(lineal_array)

In [8]:
def calculate_media_lineal_arr(array):
    length = array.shape[0]
    media = np.float64(0)
    for x in range(length):
        media += array[x]
    media = media/length
    return media

In [9]:
def lineal_to_matrix(lineal_array):
    sqrt_ = ceil(sqrt(lineal_array.shape[0]))
    media = calculate_media_lineal_arr(lineal_array)
    missing_values = sqrt_**2 - lineal_array.shape[0]
    # Complete square matrix
    for x in range(missing_values):
        lineal_array = np.append(lineal_array, media)
    
    return lineal_array.reshape([sqrt_, sqrt_])

In [10]:
def show_slice_mask(slicei, mask):
    """ Function to display images and mask """
    fig, ax = plt.subplots(1,2)
    ax[0].imshow(slicei.T,  cmap="gray", origin="lower")
    ax[0].set_title('Image')
    ax[1].imshow(mask.T, cmap="gray", origin="lower")
    ax[1].set_title('Mask')
    plt.show()

In [11]:
def glcm_properties(image):
    distancias = [1,2,3]
    #0 -> Oeste a este, np.pi/4 -> diagonal hacia abajo, np.pi/2 -> Norte a sur
    angulos = [0, np.pi/4, np.pi/2]

    # Calculate properties for just one superpixel using the
    # respective mask  
    glcm = greycomatrix(image, 
                        distances = distancias, 
                        angles = angulos,
                        symmetric=True, 
                        normed=True)
    energyResults = greycoprops(glcm, 'energy')
    #homogeneityResults = greycoprops(glcm, 'homogeneity')
    contrastResults = greycoprops(glcm, 'contrast')
    correlationResults = greycoprops(glcm, 'correlation')
    #dissimilarityResults = greycoprops(glcm, 'dissimilarity')
    
    return [contrastResults, correlationResults, energyResults]

In [32]:
def training(imgs, masks):
    height, width, num_images = imgs.shape
    classes_dict = {}
    for x in range(num_images):
        added = [0 , 0, 0]
        classes = get_vals(masks[:,:,x])
        for j in classes:
            if not int(j) in classes_dict:
                classes_dict[int(j)] = [[0, 0], [0, 0], [0, 0]]
            
            tmp, matrix_mask = apply_mask(imgs[:,:,x], masks[:,:,x], j)
            props = glcm_properties(matrix_mask.astype(np.uint8))
            for z in range(len(props)):
                if isinstance(classes_dict[int(j)][z], np.ndarray):
                    classes_dict[int(j)][z][0] += props[z]
                    classes_dict[int(j)][z][1] += 1
                else:
                    added[z] += 1
                    classes_dict[int(j)][z][0] = props[z]
                    classes_dict[int(j)][z][1] += 1
        
        if x % 10 == 0:
            print("{} imagenes procesadas".format(x))
    for id_ in classes_dict.keys():
        for z in range(len(props)):
            classes_dict[id_][z][0] = classes_dict[id_][z][0]/classes_dict[id_][z][1]
    
    return [classes_dict[id_][z][0] for x in range(len(props))]

In [33]:
def main():
    to_show = 3
    imgs = nib.load(images_filename).get_fdata()
    masks = nib.load(masks_filename).get_fdata()
    train = training(imgs, masks)
    print(train)
    """
    #for x in range(3):
    for x in [25, 26, 27]:
        print("Image no ", x)
        classes = get_vals(masks[:,:,x])
        show_slice_mask(imgs[:,:,x], masks[:,:,x])
        for j in classes:
            tmp, matrix_mask = apply_mask(imgs[:,:,x], masks[:,:,x], j)
            print("class ", j)
            print(type(matrix_mask))
            #show_slice_mask(tmp, masks[:,:,x])
            show_slice_mask(tmp, matrix_mask)
            print(glcm_properties(matrix_mask.astype(np.uint8)))
            #create_matrix(tmp)
        #print(get_prop(imgs[:,:,x].astype(np.uint8), masks[:,:,x].astype(np.uint8)))
        #get_every_mask(imgs[:,:,x], masks[:,:,x])
    """

In [34]:
main()

0 imagenes procesadas
10 imagenes procesadas
20 imagenes procesadas
30 imagenes procesadas
40 imagenes procesadas
50 imagenes procesadas
60 imagenes procesadas
70 imagenes procesadas
80 imagenes procesadas
90 imagenes procesadas
[array([[537.36343685, 665.95513548, 667.22905176],
       [659.57731933, 665.95513548, 698.54118487],
       [651.75583795, 686.23952422, 669.89021748]]), array([[537.36343685, 665.95513548, 667.22905176],
       [659.57731933, 665.95513548, 698.54118487],
       [651.75583795, 686.23952422, 669.89021748]]), array([[537.36343685, 665.95513548, 667.22905176],
       [659.57731933, 665.95513548, 698.54118487],
       [651.75583795, 686.23952422, 669.89021748]])]
